In [8]:
!pip install cvxpy pandas numpy


   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   -------------------------------------- - 1.0/1.1 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/736.4 kB ? eta -:--:--
   ---------------------------------------- 736.4/736.4 kB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.4 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.4 MB 5.0 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/8.4 MB 4.4 MB/s eta 0:00:02
   -------------- ------------------------- 3.1/8.4 MB 4.9 MB/s eta 0:00:02
   ----------------------- ---------------- 5.0/8.4 MB 5.0 MB/s eta 0:00:01
   ----------------------------- ---------- 6.3/8.4 MB 5.0 MB/s eta 0:00:01
   ---------------------------------- ----- 7.3/8.4 MB 4.9 MB/s eta 0:00:01
   ---------------------------------------  8.4/8.4 MB 5.0 MB/s eta 0:00:01
   ------------------------

In [9]:
import cvxpy as cp
import numpy as np
import pandas as pd

In [12]:
# Change the URL to download the CSV directly
url = "Companies.xlsx"

# Read the CSV with specified options
df = pd.read_excel(
    url,
    index_col=None,  # Set the first column as rownames
)

grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
df["GradeWeight"] = df["Grade"].map(grade_weight)

df.head(5)

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight
0,NextEra Energy (NEE),Utilities,0.49,B,53.47,80
1,Duke Energy (DUK),Utilities,0.38,C,57.51,70
2,Atmos Energy (ATO),Utilities,0.50,B,54.13,80
3,Avangrid (AGR),Utilities,0.34,A,-8.57,90
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90


In [ ]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')

In [14]:
# Convert grades to weights for fundamental factors, returns, and beta
grades = df["GradeWeight"].values
betas = df["Beta Value"].values
returns = df["TSR"].values
n_stocks = len(df)

# Define optimization variables
weights = cp.Variable(n_stocks)

# Risk is defined by the portfolio beta
portfolio_beta = cp.sum(cp.multiply(betas, weights))

# Expected portfolio return
portfolio_return = cp.sum(cp.multiply(returns, weights))

portfolio_grades = cp.sum(cp.multiply(grades, weights))

# Objective function: Minimize beta (risk) while maximizing returns
# We'll use a weighted objective function that can be adjusted as needed
objective = cp.Maximize(
    0.3 * portfolio_return + 0.4 * portfolio_grades + 0.2 * portfolio_beta
)

# Constraints:
constraints = [
    cp.sum(weights) == 1,  # Total weights should sum to 1
    weights >= 0,  # No shorting (long-only portfolio)
]

# Add industry diversity constraint, e.g., limit weights per industry if needed
for industry in df["Industry"].unique():
    # Create a boolean mask for the industry
    industry_mask = (df["Industry"] == industry).values
    # Apply the mask to weights
    constraints.append(cp.sum(weights[industry_mask]) <= 0.4)  # Max 20% per industry

for companys in df["Company Name"].unique():
    # Create a boolean mask for the industry
    company_mask = (df["Company Name"] == industry).values
    # Apply the mask to weights
    constraints.append(cp.sum(weights[company_mask]) <= 0.04)  # Max 4% per company

In [38]:
# "Industry",


# Formulate the problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Display results
print("Optimal weights for each stock:")
df["weight"] = weights.value
# print(df[["Industry", "weight"]])

df = df.sort_values(by="weight", ascending=False)

weighted_beta = (df["Beta Value"] * df["weight"]).sum()
weighted_tsr = (df["TSR"] * df["weight"]).sum()
print(df)
print(weighted_beta)
print(weighted_tsr)

Optimal weights for each stock:
                                Company Name          Industry  Beta Value  \
20     UnitedHealth Group Incorporated (UNH)        Healthcare        0.59   
43                                     GOOGL        Technology        1.08   
19                      JPMorgan Chase (JPM)           Finance        1.10   
47                                      SNOW        Technology        1.37   
29                         Pfizer Inc. (PFE)        Healthcare        0.62   
..                                       ...               ...         ...   
45                                      AAPL        Technology        1.29   
54                                       AVB       Real Estate        0.97   
21                   Johnson & Johnson (JNJ)        Healthcare        0.52   
42                                      META        Technology        1.26   
74  COSTCO WHOLESALE CORPORATION (XNAS:COST)  Consumer Staples        0.79   

   Grade     TSR  GradeWeight  